In [ ]:
# Basic python import
import sys
import yaml
import matplotlib.pyplot as plt

# Pytorch
from torch.utils.data import DataLoader
import optim
import torch.nn

# Our custom module
sys.path.append('../')
import data_processing
import models
import evaluation as eval
import submission as sub

# Config

Ma proposition : 
Avoir un Notebook template associé à des paramètres par défaut. 
Ensuite pour chaque expérience on le duplique, et on change les valeurs des paramètres que l'on souhaite.

In [1]:
with open('../configs/default_params.yaml') as default_params_file:
      default_params = yaml.safe_load(default_params_file)
params = default_params
params

{'experiment_title': 'write here the purpose of your experiment',
 'dataset': {'script_path': '../data_processing/A_COMPLETER.py'},
 'model': {'script_path': '../models/A_COMPLETER.py'},
 'optimizer': {'script_path': '../optimizers/adam_keras.py',
  'initial_lr': 0.0001},
 'train': {'script_path': '../train/A_COMPLETER.py',
  'batch_size': 64,
  'epochs': 1000,
  'data_augmentation': {'samplewise_center': False,
   'samplewise_std_normalization': False,
   'rotation_range': 0,
   'width_shift_range': 0.1,
   'height_shift_range': 0.1,
   'horizontal_flip': True,
   'vertical_flip': False,
   'zoom_range': 0,
   'shear_range': 0,
   'channel_shift_range': 0,
   'featurewise_center': False,
   'zca_whitening': False}},
 'evaluate': {'batch_size': 1000,
  'augmentation_factor': 32,
  'data_augmentation': {'samplewise_center': False,
   'samplewise_std_normalization': False,
   'rotation_range': 0,
   'width_shift_range': 0.15,
   'height_shift_range': 0.15,
   'horizontal_flip': True,
   

In [2]:
# Changer de paramètres
params["train"]["batch_size"] = 16
params["train"]

{'script_path': '../train/A_COMPLETER.py',
 'batch_size': 16,
 'epochs': 1000,
 'data_augmentation': {'samplewise_center': False,
  'samplewise_std_normalization': False,
  'rotation_range': 0,
  'width_shift_range': 0.1,
  'height_shift_range': 0.1,
  'horizontal_flip': True,
  'vertical_flip': False,
  'zoom_range': 0,
  'shear_range': 0,
  'channel_shift_range': 0,
  'featurewise_center': False,
  'zca_whitening': False}}

# Create Dataset and Dataloader

In [5]:
data_dir = 'data'
train_dataset = data_processing.dataset.CustomDataset(root_dir = data_dir)

train_dl = DataLoader(train_dataset, batch_size=2**12, shuffle=True)

OSError: [WinError 1455] Le fichier de pagination est insuffisant pour terminer cette opération. Error loading "c:\Users\bapti\miniconda3\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

# Train

In [ ]:
MODEL = models.Unet.Unet(num_classes=2)
lr = 1e-3
optimizer = optim.Adam(MODEL.parameters(), lr=lr)
loss = nn.MSELoss()
optimizer

In [ ]:
import train

loss_list = train.train.main_train(model=MODEL, loss_fn=loss, optimizer=optimizer, n_epochs=2, dataloader=train_dl, device="cpu")

In [ ]:
plt.plot(loss_list)
plt.title("Loss")
plt.xlabel("Epoch")

# Evaluation

Running model on validation dataset

In [ ]:
val_pred = {"val_images":[], "val_masks":[], "val_organs":[], "val_y_preds":[]}
for k, X in enumerate(train_dl):
    for val_image, val_mask, val_organ in X :
        val_pred['val_images'].append(val_image)
        val_pred['val_mask'].append(val_mask)
        val_pred['val_organ'].append(val_organ)
        pred = MODEL(val_image)
        val_pred['val_y_preds'].append(pred)

In [ ]:
IoUs, IoUsOrgans = eval.get_y_true_y_pred(val_pred)

# Plot results for different threshold on all training and get best threshold
best_threshold = eval.plot_iou_by_threshold(IoUs, 'test')

We can also look at some of the results

In [ ]:
eval.plot_validation_predictions(val_pred, best_threshold, 10)

# Submission

In [ ]:
import submission as sub

test_dataset = None
sub.make_submission(MODEL, test_dataset, best_threshold)